<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/master/notebook_util/generate_tree(drive).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

Also, check that the google account you mount the drive should have access to the shared public BUGAN drive here:
https://drive.google.com/drive/folders/1wYhB81kdrVaKf2DAgNt7tDupgg6JyJ12?usp=sharing

Please make sure the \"shared_BUGAN_folder\" above appears in the \"Shared with me\" tab

In [ ]:
#@markdown Mount google drive.
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

# Check if we have linked the folder
from pathlib import Path
if not Path("/content/drive/My Drive/").exists():
    print(
        "Drive (My Drive) not mount! \n\n"
    )

sharedDrive = Path("/content/drive/.shortcut-targets-by-id/1wYhB81kdrVaKf2DAgNt7tDupgg6JyJ12/shared_BUGAN_folder/")
if not sharedDrive.exists():
    print(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Please click the drive link above.\n"
        "\t2. Make sure the \"shared_BUGAN_folder\" appears in the \"Shared with me\" tab.\n"
    )


# Description

This notebook is used for generating 3D mesh objects based on trained runs in the wandb project "bugan/tree-gan".

User will need to specify **tree_category** and **number of samples to generate**, then the notebook will generate samples to the **export_location** specified in the panel.



---



# Instruction for specify trained model
The trained model is loaded from the saved checkpoint. 

To specify trained model:

> set **selected_tree_category**, which will load the preset *`class_index`* based on the **selected_tree_category** (unless the category is 'None').


---



# Instruction for other generate parameters
 
**selected_tree_category**:
*   The class category of the generated trees.

*   We have 22 tree_categories, each of them have around 1 to 26 tree templates. After the **selected_tree_category** is selected, one of the tree template will be selected randomly from the list.

*   However, if **selected_tree_category** is 'None', a pretrained unconditional model trained on the whole dataset will be used instead of the pretrained conditional model.

**test_num_samples**: 
*   number of samples to generate from the loaded model.

*   If the model is unconditional, it will generate **[test_num_samples]** samples.

*   If the model is conditional, it will generate **[test_num_samples]** samples of class **[class_index]**.

**generateMeshHistory**: 
*   boolean to choose whether to generate from list of history checkpoints or just the latest checkpoint.

*   In the model training, if *`history_checkpoint_frequency`* is set, then the model will save history checkpoint based on the number of epoch (e.g. save 1 checkpoint per 40 epochs). As that lots of history checkpoints will be saved, only **num_selected_checkpoint** checkpoints will be selected in the list. Enable this will make the model to generate **[test_num_samples]** samples per selected history checkpoint.

**num_selected_checkpoint**: 

*   If the **generateMeshHistory** option is True, select **num_selected_checkpoint** checkpoints in the saved history checkpoint list. 

**export_location**:
*   the export location of generated 3D meshes in the Drive.

**browser_download**:
*   if **browser_download** is True, after generating meshes into **export_location**, the generated meshes will also be zipped into a zip file, and the browser will download the zip file.
*   Note that the **browser_download** is implemented using colab `files.download()`. The download processing speed will be very slow if the zip file is **> 300 MB** (1 mesh is around 4MB).
*   Also, if **browser_download** is False, `Exception: will not download files from the browser. Terminate here.` will occur, but it is normal, and the meshes are already saved in the **export_location**.



 
















<img src="https://drive.google.com/uc?id=1KId8CncC6Ze_BsiUP-YPl-jhjSFFNCQy"></img>

In [ ]:
#@title #Specify Trained Model
#@markdown - choose which option to specify trained model

# #@markdown Choice 1: wandb run_id and to select checkpoint file (leave empty if you want to use preset model option 2)
# #@markdown - set `"run_id"` to resume a run (for example: `u9imsvva`)
# #@markdown - The id of the current run is shown below in the cell with `wandb.init()` or you can find it in the wandb web UI.
# resume_id = "3mzlr9u3" #@param {type:"string"}
# #@markdown - select class index for conditional model (will be ignored for choice 2 or if the model is unconditional)
# class_index = 3 #@param {type:"integer"}
#(please make sure resume_id is empty)
#@markdown generate tree from preset models based on class 
#@markdown - Select tree class: load the preset run_id and class_index based on the selected_tree_category 
selected_tree_category = "None"    #@param ['double_trunk', 'formal_upright', 'friedrich', 'group', 'informal_upright', 'leaning', 'mustard_reaching', 'mustard_sapling', 'banyan', 'maple', 'old', 'pine', 'tall_straight', 'raft', 'semi_cascade', 'chen_lin', 'constable', 'holten', 'mondrian', 'schiele', 'windswept', 'zan_gentlemen', 'None']

class_categories_dict = {'double_trunk': ['double_trunk_1', 'double_trunk_2'],
                         'formal_upright': ['formal_upright_1', 'formal_upright_2'],
                         'friedrich': ['friedrich_1','friedrich_2','friedrich_3','sept_friedrich_4', 'sept_friedrich_5'],
                         'group': ['group_1', 'group_2'],
                         'informal_upright': ['informal_upright_1', 'double_trunk_2'],
                         'leaning': ['leaning_1', 'leaning_2'],
                         'mustard_reaching': ['mustard_reaching_1'],
                         'mustard_sapling': ['mustard_sapling_1', 'mustard_sapling_2'],
                         'banyan': ['pn_banyan_1'],
                         'maple': ['pn_maple_1'],
                         'old': ['pn_old_1'],
                         'pine': ['pn_pine_1', 'pn_pine_2', 'pn_pine_3','pn_pine_4','pn_pine_5','pn_pine_6'],
                         'tall_straight': ['pn_tall_straight', 'pn_tall_straight_old'],
                         'raft': ['raft_1', 'raft_2'],
                         'semi_cascade': ['semi_cascade_1', 'semi_cascade_2'],
                         'chen_lin': ['sept_chen_lin_1', 'sept_chen_lin_2', 'sept_chen_lin_3'],
                         'constable': ['sept_constable_1'],
                         'holten': ['sept_holten_a', 'sept_holten_b', 'sept_holten_c','sept_holten_d','sept_holten_e','sept_holten_f', 'sept_holten_g', 'sept_holten_h' , 'sept_holten_i', 'sept_holten_j', 'sept_holten_k', 'sept_holten_l', 'sept_holten_m', 'sept_holten_n', 'sept_holten_o', 'sept_holten_p', 'sept_holten_q', 'sept_holten_r', 'sept_holten_s', 'sept_holten_t', 'sept_holten_u', 'sept_holten_v', 'sept_holten_w', 'sept_holten_x', 'sept_holten_y', 'sept_holten_z'],
                         'mondrian': ['sept_mondrian_1', 'sept_mondrian_2', 'sept_mondrian_3'],
                         'schiele': ['sept_schiele_1', 'sept_schiele_2', 'sept_schiele_3'],
                         'windswept': ['windswept_1', 'windswept_2'],
                         'zan_gentlemen': ['zan_gentlemen_1', 'zan_gentlemen_2', 'zan_gentlemen_3', 'zan_gentlemen_4', 'zan_gentlemen_5'],
                         }

preset_class_index_dict = {
    "double_trunk_1": 68,
    "double_trunk_2": 3,
    "formal_upright_1": 59,
    "formal_upright_2": 20,
    "friedrich_1": 41,
    "friedrich_2": 31,
    "friedrich_3": 51,
    "group_1": 22,
    "group_2": 12,
    "informal_upright_1": 60,
    "informal_upright_2": 61,
    "leaning_1": 39,
    "leaning_2": 50,
    "mustard_reaching_1": 24,
    "mustard_sapling_1": 1,
    "mustard_sapling_2": 73,
    "pn_banyan_1": 37,
    "pn_maple_1": 64,
    "pn_old_1": 14,
    "pn_pine_1": 58,
    "pn_pine_2": 28,
    "pn_pine_3": 19,
    "pn_pine_4": 65,
    "pn_pine_5": 55,
    "pn_pine_6": 53,
    "pn_tall_straight": 40,
    "pn_tall_straight_old": 69,
    "raft_1": 63,
    "raft_2": 62,
    "semi_cascade_1": 23,
    "semi_cascade_2": 6,
    "sept_chen_lin_1": 16,
    "sept_chen_lin_2": 72,
    "sept_chen_lin_3": 66,
    "sept_constable_1": 29,
    "sept_friedrich_4": 35,
    "sept_friedrich_5": 47,
    "sept_holten_a": 11,
    "sept_holten_b": 38,
    "sept_holten_c": 70,
    "sept_holten_d": 67,
    "sept_holten_e": 25,
    "sept_holten_f": 32,
    "sept_holten_g": 5,
    "sept_holten_h": 18,
    "sept_holten_i": 46,
    "sept_holten_j": 7,
    "sept_holten_k": 17,
    "sept_holten_l": 74,
    "sept_holten_m": 8,
    "sept_holten_n": 48,
    "sept_holten_o": 36,
    "sept_holten_p": 34,
    "sept_holten_q": 75,
    "sept_holten_r": 21,
    "sept_holten_s": 9,
    "sept_holten_t": 10,
    "sept_holten_u": 42,
    "sept_holten_v": 27,
    "sept_holten_w": 44,
    "sept_holten_x": 52,
    "sept_holten_y": 30,
    "sept_holten_z": 33,
    "sept_mondrian_1": 26,
    "sept_mondrian_2": 49,
    "sept_mondrian_3": 13,
    "sept_schiele_1": 0,
    "sept_schiele_2": 43,
    "sept_schiele_3": 2,
    "windswept_1": 15,
    "windswept_2": 57,
    "zan_gentlemen_1": 71,
    "zan_gentlemen_2": 56,
    "zan_gentlemen_3": 54,
    "zan_gentlemen_4": 4,
    "zan_gentlemen_5": 45,
}
from numpy.random import randint

if selected_tree_category == 'None':
    preset_model_checkpoint = "checkpoint_uncond.ckpt"
    selected_model = "VAEGAN"
    # this index will be ignored
    selected_tree_index = 0 
    print("selected_tree_category:", selected_tree_category)
else:
    #from selected_tree_category, find tree_class_list
    tree_class_list = class_categories_dict[selected_tree_category]
    selected_tree_class_index = randint(len(tree_class_list))
    selected_tree_class = tree_class_list[selected_tree_class_index]
    #from selected_tree_class, find preset_index
    selected_tree_index = preset_class_index_dict[selected_tree_class]

    print("selected_tree_category:", selected_tree_category)
    print("selected_tree_class_template:", selected_tree_class)
    print("selected_tree_index:", selected_tree_index)

    preset_model_checkpoint = "checkpoint.ckpt"
    selected_model = "CVAEGAN"

project_name = "tree-gan"

In [ ]:
#@title ##other generate parameters

#@markdown number of samples to generate from the loaded model.
test_num_samples = 10    #@param {type:"integer"}

# #@markdown OPTIONAL: boolean to choose whether to generate from list of history checkpoints or just the latest checkpoint.
# generateMeshHistory = False #@param {type:"boolean"}
# #@markdown OPTIONAL: If generateMeshHistory enabled, select "num_selected_checkpoint" checkpoints in the saved history checkpoint list.
# num_selected_checkpoint = 4 #@param {type:"integer"}

#@markdown Enter export location (folder/directory).   
export_location = f"/content/drive/MyDrive/exportTree/{selected_tree_category}" #@param {type:"string"}

#@markdown whether to zip file and download with browser
browser_download = False #@param {type:"boolean"}

print("test_num_samples:", test_num_samples)
print("export_location:", export_location)

# To just load the model, no edits should be required in any cells below.

In [ ]:
#@markdown package and functions

import os
import sys
import subprocess
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from pathlib import Path
os.environ["WANDB_MODE"] = "dryrun"

%pip install --upgrade git+https://github.com/buganart/BUGAN.git#egg=bugan

run_path = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/"

from bugan.trainPL import _get_models
from bugan.functionsPL import netarray2mesh

def install_bugan_package(rev_number=None):
    if rev_number:
        subprocess.check_call(
            [
                sys.executable,
                "-m",
                "pip",
                "install",
                "--upgrade",
                f"git+https://github.com/buganart/BUGAN.git@{rev_number}#egg=bugan",
            ]
        )
    else:
        subprocess.check_call(
            [
                sys.executable,
                "-m",
                "pip",
                "install",
                "--upgrade",
                "git+https://github.com/buganart/BUGAN.git#egg=bugan",
            ]
        )

# load model
def generateFromCheckpoint(selected_model, ckpt_filePath, class_index=None, num_samples=1, package_rev_number=None, output_file_name_dict = {}):
    MODEL_CLASS = _get_models(selected_model)
    try:
        # restore bugan version
        install_bugan_package(rev_number=package_rev_number)
        model = MODEL_CLASS.load_from_checkpoint(ckpt_filePath)
    except Exception as e:
        print(e)
        # try newest bugan version
        install_bugan_package()
        model = MODEL_CLASS.load_from_checkpoint(ckpt_filePath)

    model = model.eval()#.to(device)
    try:
        #assume conditional model
        sample_trees = model.generate_tree(c=class_index, num_trees=test_num_samples)
        output_file_name_dict["class"] = class_index
    except Exception as e:
        print(e)
        print("generate with class label does not work. Now generate without label")
        #assume unconditional model
        sample_trees = model.generate_tree(num_trees=test_num_samples)
    
    save_filename_header = ""
    for k,v in output_file_name_dict.items():
        save_filename_header = save_filename_header + f"_{str(k)}_{str(v)}"

    for n in range(num_samples):
        sample_tree_array = sample_trees[n]
        voxelmesh = netarray2mesh(sample_tree_array)
        save_filename = f"sample_{n}{save_filename_header}.obj"
        if export_location:
            export_path = export_location / save_filename
            voxelmesh.export(file_obj=export_path, file_type="obj")
        #if zip, save also to temp_location 
        if browser_download:
            temp_path = temp_location / save_filename
            voxelmesh.export(file_obj=temp_path, file_type="obj")

output.clear()
print('ok!')

In [ ]:
#@markdown generate samples
temp_location = Path("/tmp/generated/")
temp_location.mkdir(parents=True, exist_ok=True)
# copy files to export location
if export_location:
    # create directory
    export_location = Path(export_location)
    export_location.mkdir(parents=True, exist_ok=True)

sharedFolder = sharedDrive / "tree/checkpoint"
ckpt_file = sharedFolder / preset_model_checkpoint
generateFromCheckpoint(selected_model, ckpt_file, selected_tree_index, test_num_samples, None, {})


#zip files in the directory
if not browser_download:
    raise Exception("will not download files from the browser. Terminate here.")

!zip -r /tmp/file.zip /tmp/generated/
from google.colab import files
files.download("/tmp/file.zip")

print("ok!")